# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,49.37,86,66,5.23,NZ,1649507113
1,1,gedo,9.0167,37.4500,80.37,16,20,5.44,ET,1649507113
2,2,deputatskiy,69.3000,139.9000,-11.36,100,100,3.06,RU,1649507113
3,3,cayambe,0.0500,-78.1333,49.26,86,100,3.11,EC,1649507113
4,4,moba,33.2288,105.0179,58.37,56,48,4.79,CN,1649507113


In [20]:
len(city_data_df)

571

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [22]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Create a list of locations that are between 70 and 80 deg with no cloud cover and wind speed less than 10 mph. drop non
# qualifiying locations and create a perfect_place list
perfect_place = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
perfect_place


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,kahului,20.8947,-156.4700,74.84,76,0,7.00,US,1649507118
88,88,victoria,22.2855,114.1577,75.38,66,0,8.01,HK,1649507039
94,94,santa fe,-31.6333,-60.7000,72.00,76,0,5.01,AR,1649507123
457,457,puerto escondido,15.8500,-97.0667,74.23,88,0,5.75,MX,1649507162
483,483,tarsus,36.9188,34.8784,79.18,63,0,9.10,TR,1649507164
509,509,ziarat,30.3823,67.7282,72.82,8,0,9.53,PK,1649507167


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = perfect_place[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
52,kahului,US,20.8947,-156.4700,
88,victoria,HK,22.2855,114.1577,
94,santa fe,AR,-31.6333,-60.7000,
457,puerto escondido,MX,15.8500,-97.0667,
483,tarsus,TR,36.9188,34.8784,
509,ziarat,PK,30.3823,67.7282,


In [25]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
52,kahului,US,20.8947,-156.4700,Maui Seaside Hotel
88,victoria,HK,22.2855,114.1577,Mini Hotel Central
94,santa fe,AR,-31.6333,-60.7000,Conquistador Hotel
457,puerto escondido,MX,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
483,tarsus,TR,36.9188,34.8784,Osmanli Marco Pasha Hotel
509,ziarat,PK,30.3823,67.7282,Pakiza Guest House ziarat


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))